# Multi-Echo Denoising with `tedana`

In this analysis tutorial, we will use `tedana` {cite:p}`DuPre2021` to perform multi-echo denoising.

Specifically, we will use {py:func}`tedana.workflows.tedana_workflow`.

In [1]:
import os
import matplotlib.pyplot as plt
from glob import glob

import numpy as np
import pandas as pd
from nilearn import image, plotting
from tedana import workflows
from IPython.display import display, HTML
import json
from pprint import pprint

from repo2data.repo2data import Repo2Data

# Install the data if running locally, or points to cached data if running on neurolibre
DATA_REQ_FILE = os.path.join("../binder/data_requirement.json")

# Download data
repo2data = Repo2Data(DATA_REQ_FILE)
data_path = repo2data.install()
data_path = os.path.abspath(os.path.join(data_path[0], "data"))

/opt/hostedtoolcache/Python/3.7.12/x64/lib/python3.7/site-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)
Module `duecredit` not successfully imported due to "No module named 'duecredit'". Package functionality unaffected.


Failed to import duecredit due to No module named 'duecredit'


---- repo2data starting ----
/opt/hostedtoolcache/Python/3.7.12/x64/lib/python3.7/site-packages/repo2data
Config from file :
../binder/data_requirement.json
Destination:
./../data/multi-echo-data-analysis

Info : ./../data/multi-echo-data-analysis already downloaded


In [2]:
func_dir = os.path.join(data_path, "sub-04570/func/")
data_files = [
    os.path.join(func_dir, "sub-04570_task-rest_echo-1_space-scanner_desc-partialPreproc_bold.nii.gz"),
    os.path.join(func_dir, "sub-04570_task-rest_echo-2_space-scanner_desc-partialPreproc_bold.nii.gz"),
    os.path.join(func_dir, "sub-04570_task-rest_echo-3_space-scanner_desc-partialPreproc_bold.nii.gz"),
    os.path.join(func_dir, "sub-04570_task-rest_echo-4_space-scanner_desc-partialPreproc_bold.nii.gz"),
]
echo_times = [12., 28., 44., 60.]
mask_file = os.path.join(func_dir, "sub-04570_task-rest_space-scanner_desc-brain_mask.nii.gz")
confounds_file = os.path.join(func_dir, "sub-04570_task-rest_desc-confounds_timeseries.tsv")

out_dir = os.path.join(data_path, "tedana")

In [3]:
workflows.tedana_workflow(
    data_files,
    echo_times,
    out_dir=out_dir,
    mask=mask_file,
    prefix="sub-04570_task-rest_space-scanner",
    fittype="curvefit",
    tedpca="mdl",
)

INFO     tedana:tedana_workflow:454 Using output directory: /home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/data/tedana


INFO     tedana:tedana_workflow:467 Loading input data: ['/home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/data/sub-04570/func/sub-04570_task-rest_echo-1_space-scanner_desc-partialPreproc_bold.nii.gz', '/home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/data/sub-04570/func/sub-04570_task-rest_echo-2_space-scanner_desc-partialPreproc_bold.nii.gz', '/home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/data/sub-04570/func/sub-04570_task-rest_echo-3_space-scanner_desc-partialPreproc_bold.nii.gz', '/home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/data/sub-04570/func/sub-04570_task-rest_echo-4_space-scanner_desc-partialPreproc_bold.nii.gz']


INFO     io:__init__:106 Generating figures directory: /home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/data/tedana/figures


INFO     tedana:tedana_workflow:536 Using user-defined mask


INFO     tedana:tedana_workflow:584 Computing T2* map


...

INFO     io:writeresults:508 Writing Z-normalized spatial component maps: /home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/data/tedana/sub-04570_task-rest_space-scanner_desc-ICAAccepted_stat-z_components.nii.gz


INFO     tedana:tedana_workflow:861 Making figures folder with static component maps and timecourse plots.


INFO     io:denoise_ts:374 Variance explained by decomposition: 92.18%


INFO     tedana:tedana_workflow:890 Generating dynamic report


INFO     tedana:tedana_workflow:893 Workflow completed


In [4]:
out_files = sorted(glob(os.path.join(out_dir, "*")))
out_files = [os.path.basename(f) for f in out_files]
print("\n".join(out_files))

figures
report.txt
sub-04570_task-rest_space-scanner_S0map.nii.gz
sub-04570_task-rest_space-scanner_T2starmap.nii.gz
sub-04570_task-rest_space-scanner_dataset_description.json
sub-04570_task-rest_space-scanner_desc-ICAAccepted_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAAccepted_stat-z_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICA_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICA_decomposition.json
sub-04570_task-rest_space-scanner_desc-ICA_mixing.tsv
sub-04570_task-rest_space-scanner_desc-ICA_stat-z_components.nii.gz
sub-04570_task-rest_space-scanner_desc-PCA_decomposition.json
sub-04570_task-rest_space-scanner_desc-PCA_metrics.json
sub-04570_task-rest_space-scanner_desc-PCA_metrics.tsv
sub-04570_task-rest_space-scanner_desc-PCA_mixing.tsv
sub-04570_task-rest_space-scanner_desc-PCA_stat-z_components.nii.gz
sub-04570_task-rest_space-scanner_desc-adaptiveGoodSignal_mask.nii.gz
sub-04570_task-rest_space-scanner_desc-optcomAccepted_bold.nii.gz
sub

In [5]:
metrics = pd.read_table(os.path.join(out_dir, "sub-04570_task-rest_space-scanner_desc-tedana_metrics.tsv"))
metrics

,Component,kappa,rho,variance explained,normalized variance explained,countsigFT2,countsigFS0,dice_FT2,dice_FS0,countnoise,signal-noise_t,signal-noise_p,d_table_score,optimal sign,kappa ratio,d_table_score_scrub,classification,rationale
0,ICA_00,30.454429,34.549600,0.675289,0.008084,1324,2119,0.482574,0.473596,1180,-0.603544,5.467640e-01,19.4,-1,1.864353,NaN,rejected,I002;I003;I005
1,ICA_01,12.037751,19.574322,0.230989,0.003451,109,192,0.000000,0.083551,1008,-8.026861,8.114305e-14,37.0,-1,1.613377,NaN,rejected,I002;I003
2,ICA_02,26.828837,20.843478,1.193384,0.011905,1295,353,0.337510,0.154506,1399,0.000000,0.000000e+00,23.9,1,3.739965,12.6,rejected,I010
3,ICA_03,33.999051,38.449963,1.647885,0.015925,594,806,0.371651,0.373913,1137,3.035790,5.854269e-03,17.4,1,4.075201,NaN,rejected,I002;I003;I004
4,ICA_04,16.010697,14.681236,0.266328,0.003979,311,791,0.000000,0.345646,1310,-2.036624,4.290170e-02,37.0,1,1.398605,NaN,rejected,I003
5,ICA_05,15.108999,26.581132,0.453214,0.007364,147,347,0.307167,0.433140,824,0.291266,7.709317e-01,26.6,-1,2.522066,NaN,rejected,I002;I003
6,ICA_06,46.226736,11.770721,0.314894,0.004472,1848,384,0.578977,0.220979,999,12.847665,1.349604e-32,7.8,-1,0.572744,6.0,accepted,NaN
7,ICA_07,19.174667,21.564404,0.333725,0.003534,302,448,0.224490,0.354430,1256,0.751410,4.540167e-01,28.5,-1,1.463356,NaN,rejected,I002;I003
8,ICA_08,25.882636,14.985862,1.080261,0.007775,1054,199,0.318403,0.005089,1336,-0.593940,5.568770e-01,26.0,1,3.509208,NaN,rejected,I005
9,ICA_09,16.406118,17.135616,0.203201,0.002685,258,455,0.000000,0.162637,1465,0.000000,0.000000e+00,37.5,-1,1.041381,NaN,rejected,I002;I003


In [6]:
with open(os.path.join(out_dir, "sub-04570_task-rest_space-scanner_desc-tedana_metrics.json"), "r") as fo:
    data = json.load(fo)

first_five_keys = list(data.keys())[:5]
reduced_data = {k: data[k] for k in first_five_keys}
pprint(reduced_data)

{'Component': {'Description': 'The unique identifier of each component. This '
                              'identifier matches column names in the mixing '
                              'matrix TSV file.',
               'LongName': 'Component identifier'},
 'classification': {'Description': 'Classification from the manual '
                                   'classification procedure.',
                    'Levels': {'accepted': 'A BOLD-like component included in '
                                           'denoised and high-Kappa data.',
                               'ignored': 'A low-variance component included '
                                          'in denoised, but excluded from '
                                          'high-Kappa data.',
                               'rejected': 'A non-BOLD component excluded from '
                                           'denoised and high-Kappa data.'},
                    'LongName': 'Component classification'},
 'countnoise': 

In [7]:
df = pd.DataFrame.from_dict(data, orient="index")
df = df.fillna("n/a")
display(HTML(df.to_html()))

,Description,LongName,Levels,Units
Component,The unique identifier of each component. This identifier matches column names in the mixing matrix TSV file.,Component identifier,n/a,n/a
classification,Classification from the manual classification procedure.,Component classification,"{'accepted': 'A BOLD-like component included in denoised and high-Kappa data.', 'ignored': 'A low-variance component included in denoised, but excluded from high-Kappa data.', 'rejected': 'A non-BOLD component excluded from denoised and high-Kappa data.'}",n/a
countnoise,"Number of 'noise' voxels (voxels highly weighted for component, but not from clusters) from each component.",Noise voxel count,n/a,voxel
countsigFS0,Number of significant voxels from the cluster-extent thresholded S0 model F-statistic map for each component.,S0 model F-statistic map significant voxel count,n/a,voxel
countsigFT2,Number of significant voxels from the cluster-extent thresholded T2 model F-statistic map for each component.,T2 model F-statistic map significant voxel count,n/a,voxel
d_table_score,"Summary score compiled from five metrics, with smaller values (i.e., higher ranks) indicating more BOLD dependence and less noise.",Decision table score,n/a,arbitrary
d_table_score_scrub,"Summary score compiled from five metrics and computed from a subset of components, with smaller values (i.e., higher ranks) indicating more BOLD dependence and less noise.",Updated decision table score,n/a,arbitrary
dice_FS0,Dice value of cluster-extent thresholded maps of S0-model betas and F-statistics.,S0 model beta map-F-statistic map Dice similarity index,n/a,arbitrary
dice_FT2,Dice value of cluster-extent thresholded maps of T2-model betas and F-statistics.,T2 model beta map-F-statistic map Dice similarity index,n/a,arbitrary
kappa,"A pseudo-F-statistic indicating TE-dependence of the component. This metric is calculated by computing fit to the TE-dependence model at each voxel, and then performing a weighted average based on the voxel-wise weights of the component.",Kappa,n/a,arbitrary
